# ISM Lecture 8 week 09 Part 3 Solutions

This content is authored by Maria Boutchkova for use in the University of Edinbugh Business School Investment and Securities Markets course in Autumn 2020.

Make sure to have watched the videos preceeding this Notebook and have covered the slides. Detailed explanations in the assigned textbook chapters.

This lesson covers:

* Fama-MacBeth test of CAPM as in BKM chapter 13 section 1 performed on individual assets instead of test portfolios.

The first computational cell below (with In [ ] in front) contains the solution. Go over the command lines, make sure they make sense to you, click inside the cell, it should become surrounded by a green rectangle, press Esc - the rectangle will become blue, now press Shift+Enter - this will execute the cell and produce the results beneath it.

To remove all output in the notebook and start again, go to the Kernel tab above, select Restart and Clear Output.

In this notebook we use the functionality of Stata. If you want to explore its full documetation, see here: https://www.stata.com/bookstore/getting-started-windows/

The comment sign in Stata is * not #

In [ ]:
* again import the data we prepared in python
import delimited data.csv, varnames(1) clear

In [ ]:
%browse

# Solved Problem 1: Multiple time-series regressions in a loop

Now we are going to loop over all stocks, regress each of them on the market return and save three outputs from each regression: alpha, beta and the root MSE.

In [ ]:
* first we shall save all variable names of the stocks without snp in a local macro using the convenient command in stata unab
unab stocks: amexlng - xtrarwe
* check to see if the what we want is saved in the local macro stocks
di "`stocks'"

In [ ]:
%help unab

In [ ]:
* now we shall run all the regressions and save the 3 numbers in the last 3 lines below the data

* add 3 more empty observations
* in Stata the total number of observations in a dataset in memory is _N
local new_obs = _N + 3
set obs `new_obs'

In [ ]:
* set up the loop
foreach v of local stocks {
    qui reg `v' snp
    qui replace `v' = _b[_cons] in -3
    qui replace `v' = _b[snp] in -2
    qui replace `v' = e(rmse) in -1
}

In [ ]:
%tail

# Solved Problem 2: Compute averages and save in the last row

We are again going to loop over all stocks, including the mkt return and compute their means and store them in the bottom of the data.

In [ ]:
local new_obs = _N + 1
set obs `new_obs'

In [ ]:
* set up the loop, this time we want all variables no need to exclude snp
* the option meanonly speeds up the code and works for us since we only need the mean
unab stocks: snp - xtrarwe
foreach v of local stocks {
    qui sum `v' in 1/-5, meanonly
    qui replace `v' = r(mean) in -1
}

In [ ]:
%tail

# Solved Problem 3: Run the second-pass regression and plot the results

We are now ready to regress the average return on the betas, betas squared and the root MSE of all time-series regressions. But first we need to rearange the last 4 rows of the data into columns

In [ ]:
drop date
keep in -4/l
xpose, clear
rename v1 alpha
rename v2 beta
rename v3 rmse
rename v4 ave

In [ ]:
%browse

In [ ]:
* save the average mkt excess return in a local macro
qui sum ave in 1, meanonly
local ave_mkt = r(mean)
drop in 1
gen beta_sq = beta*beta
reg ave beta beta_sq rmse
di "`ave_mkt'"

In [ ]:
* plot the data and the linear fit of ave on beta
scatter ave beta || lfit ave beta 

# Practice Problem: Perform all thse steps for your own data

Now perform these same steps for your data that you saved in part 1 in the csv file my_data.

In [ ]:
import delimited my_data.csv, varnames(1) clear

In [ ]:
%browse

In [ ]:
* change the below to your own tickers
unab stocks: nyseaig - nasdaqgszbra
* check to see if the what we want is saved in the local macro stocks
di "`stocks'"

In [ ]:
* add 3 more empty observations
local new_obs = _N + 3
set obs `new_obs'

In [ ]:
* set up the loop
foreach v of local stocks {
    qui reg `v' snp
    qui replace `v' = _b[_cons] in -3
    qui replace `v' = _b[snp] in -2
    qui replace `v' = e(rmse) in -1
}

In [ ]:
%tail

In [ ]:
local new_obs = _N + 1
set obs `new_obs'

In [ ]:
* change the below to your own last ticker
unab stocks: snp - nasdaqgszbra
foreach v of local stocks {
    qui sum `v' in 1/-5, meanonly
    qui replace `v' = r(mean) in -1
}

In [ ]:
drop date
keep in -4/l
xpose, clear
rename v1 alpha
rename v2 beta
rename v3 rmse
rename v4 ave

In [ ]:
* save the average mkt excess return in a local macro and drop the first obs
qui sum ave in 1, meanonly
local ave_mkt = r(mean)
drop in 1

In [ ]:
* generate the squared beta
gen beta_sq = beta*beta
* run the second pass regression
reg ave beta beta_sq rmse

In [ ]:
* plot the data and the linear fit of ave on beta
scatter ave beta || lfit ave beta